In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model


2023-09-27 15:20:49.759625: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 15:20:49.817547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-27 15:20:49.819009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 15:20:51.115798: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
for folder in os.listdir('img'):
    for foto in os.listdir(f'img/{folder}'):
        print(f'img/{folder}/{foto}')

img/dzikri/dzikri.18.png
img/dzikri/dzikri.12.png
img/dzikri/dzikri.5.png
img/dzikri/dzikri.19.png
img/dzikri/dzikri.4.png
img/dzikri/dzikri.7.png
img/dzikri/dzikri.8.png
img/dzikri/dzikri.10.png
img/dzikri/dzikri.11.png
img/dzikri/dzikri.44.png
img/dzikri/dzikri.6.png
img/dzikri/dzikri.13.png
img/dzikri/dzikri.9.png
img/dzikri/dzikri.3.png
img/dzikri/dzikri.16.png
img/dzikri/dzikri.17.png
img/dzikri/dzikri.1.png
img/dzikri/dzikri.2.png
img/dzikri/dzikri.15.png
img/dzikri/dzikri.20.png
img/diva/diva.4.png
img/diva/diva.1.png
img/diva/diva.2.png
img/diva/diva.14.png
img/diva/diva.6.png
img/diva/diva.17.png
img/diva/diva.20.png
img/diva/diva.3.png
img/diva/diva.7.png
img/diva/diva.12.png
img/diva/diva.15.png
img/diva/diva.18.png
img/diva/diva.8.png
img/diva/diva.16.png
img/diva/diva.5.png
img/diva/diva.9.png
img/diva/diva.11.png
img/diva/diva.19.png
img/diva/diva.10.png
img/diva/diva.13.png
img/uki/uki.19.png
img/uki/uki.14.png
img/uki/uki.2.png
img/uki/uki.5.png
img/uki/uki.4.png
img/uk

In [3]:
face_cascade = cv2.CascadeClassifier('cascade/face_detect.xml')
def load_images_from_folder(folder):
    images = []
    labels = []
    label_encoder = LabelEncoder()
    for subfolder in os.listdir(folder):
        # subfolder_path = os.path.join(folder, subfolder)
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Ubah ke skala abu-abu jika perlu
                wajah = face_cascade.detectMultiScale(img, 1.3, 5)
                for (x, y, w, h) in wajah:
                    img = img[y:y+h, x:x+w] / 255.0
                    img = cv2.resize(img, (128, 128))
                    images.append(img)
                    labels.append(subfolder)  # Label adalah nama subfolder
    print(labels)
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    print(labels)
    return images, labels

dataset_folder = 'dataset/'
images, labels = load_images_from_folder(dataset_folder)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Inisialisasi model
model = models.Sequential()

# Tambahkan lapisan konvolusi pertama
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Tambahkan lapisan konvolusi kedua
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Tambahkan lapisan konvolusi ketiga (opsional)
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Lapisan Flatten untuk mengubah matriks menjadi vektor
model.add(layers.Flatten())

# Tambahkan lapisan fully connected pertama
model.add(layers.Dense(512, activation='relu'))

# Tambahkan lapisan fully connected kedua (output layer)
num_classes = 4
model.add(layers.Dense(num_classes, activation='softmax'))  # Ganti num_classes dengan jumlah kelas identitas wajah

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Jika ini adalah masalah klasifikasi
              metrics=['accuracy'])

# Tampilkan ringkasan model
model.summary()
model.save('nama_model.h5')

# Konversi label menjadi one-hot encoding
label_binarizer = LabelBinarizer()
y_train_encoded = label_binarizer.fit_transform(y_train)
y_test_encoded = label_binarizer.transform(y_test)

desired_size = (128, 128)

# Resizing gambar-gambar dalam dataset
X_train_resized = [cv2.resize(img, desired_size) for img in X_train]
X_test_resized = [cv2.resize(img, desired_size) for img in X_test]


# Konversi data gambar dan label menjadi array NumPy
X_train_array = np.array(X_train_resized)
X_test_array = np.array(X_test_resized)

# Reshape data untuk memenuhi format yang diperlukan oleh model (sesuai dengan input_shape)
X_train_array = X_train_array.reshape(-1, 128, 128, 1)  # Ganti dimensi sesuai dengan ukuran gambar
X_test_array = X_test_array.reshape(-1, 128, 128, 1)

# Normalisasi nilai piksel ke rentang [0, 1]
X_train_array = X_train_array / 255.0
X_test_array = X_test_array / 255.0

# Pelatihan model
history = model.fit(
    X_train_array, y_train_encoded,
    epochs=10,  # Ganti jumlah epoch sesuai kebutuhan
    batch_size=32,  # Ganti ukuran batch sesuai kebutuhan
    validation_data=(X_test_array, y_test_encoded)
)

# Evaluasi model
loss, accuracy = model.evaluate(X_test_array, y_test_encoded)
print(f"Loss: {loss}, Accuracy: {accuracy}")


# Prediksi kelas untuk dataset pengujian
y_pred = model.predict(X_test_array)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai prediksi
y_pred_classes = np.argmax(y_pred, axis=1)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai label sebenarnya
y_true = np.argmax(y_test_encoded, axis=1)

# Menghitung akurasi
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Akurasi: {accuracy}")

# Membuat laporan klasifikasi
classification_rep = classification_report(y_true, y_pred_classes)
print("Laporan Klasifikasi:")
print(classification_rep)

# Menghitung dan menampilkan matriks kebingungan (confusion matrix)
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
print("Matriks Kebingungan:")
print(confusion_mtx)


# Import library yang diperlukan

# Memuat model yang sudah dilatih
model = load_model('nama_model.h5')  # Ganti 'nama_model.h5' dengan nama model yang sudah Anda simpan

# Membaca gambar yang akan diprediksi
input_image_path = 'imgtest/diva/diva.21.png'  # Ganti dengan path gambar input Anda
input_image = cv2.imread(input_image_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)  # Ubah ke skala abu-abu jika diperlukan
input_image = cv2.resize(input_image, (128, 128))  # Sesuaikan dengan ukuran yang digunakan saat pelatihan
input_image = input_image / 255.0  # Normalisasi

# Lakukan inferensi
input_image = np.expand_dims(input_image, axis=0)  # Menambahkan dimensi batch
predictions = model.predict(input_image)

# Mengambil kelas dengan probabilitas tertinggi sebagai prediksi
predicted_class = np.argmax(predictions, axis=1)

# Konversi indeks kelas menjadi label atau hasil yang sesuai dengan aplikasi Anda
# Contoh: jika Anda memiliki dictionary yang memetakan indeks ke label
# class_labels = {0: 'Label 0', 1: 'Label 1', 2: 'Label 2'} 
 # Ganti dengan label-label Anda

class_labels = {}
for i, isi in enumerate(os.listdir('img/')):
    class_labels[i] = isi

print(class_labels)
predicted_label = class_labels[predicted_class[0]]

# Cetak hasil prediksi
print(f'Hasil prediksi: {predicted_label}')



[]


ValueError: zero-size array to reduction operation maximum which has no identity

In [5]:
face_cascade = cv2.CascadeClassifier('cascade/face_detect.xml')

def load_images_from_folder(folder):
    images = []
    labels = []
    label_encoder = LabelEncoder()
    for subfolder in os.listdir(folder):
        # subfolder_path = os.path.join(folder, subfolder)
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Ubah ke skala abu-abu jika perlu
                wajah = face_cascade.detectMultiScale(img, 1.3, 5)
                for (x, y, w, h) in wajah:
                    img = cv2.resize(img[y:y+h, x:x+w], (100, 100))
                    img = img / 255.0
    #                 cv2.imshow("gambar", img)
    #                 cv2.waitKey(0)
    # cv2.destroyAllWindows()
                    images.append(img)
                    labels.append(subfolder)  # Label adalah nama subfolder
    print(labels)
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    print(labels)
    return images, labels

dataset_folder = 'img/'
images, labels = load_images_from_folder(dataset_folder)

# load_images_from_folder(dataset_folder)


['dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'diva', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'uki', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini', 'arini']
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [10]:
# Inisialisasi model
model = models.Sequential()

# Tambahkan lapisan konvolusi pertama
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Tambahkan lapisan konvolusi kedua
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Tambahkan lapisan konvolusi ketiga (opsional)
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Lapisan Flatten untuk mengubah matriks menjadi vektor
model.add(layers.Flatten())

# Tambahkan lapisan fully connected pertama
model.add(layers.Dense(512, activation='relu'))

# Tambahkan lapisan fully connected kedua (output layer)
num_classes = 4
model.add(layers.Dense(num_classes, activation='softmax'))  # Ganti num_classes dengan jumlah kelas identitas wajah

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Jika ini adalah masalah klasifikasi
              metrics=['accuracy'])

# Tampilkan ringkasan model
model.summary()
model.save('nama_model.h5')


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 10, 128)      

/home/ahmad/opencvPro/App-BPH-Manage/venv/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Konversi label menjadi one-hot encoding
label_binarizer = LabelBinarizer()
y_train_encoded = label_binarizer.fit_transform(y_train)
y_test_encoded = label_binarizer.transform(y_test)

desired_size = (100, 100)

# Resizing gambar-gambar dalam dataset
X_train_resized = [cv2.resize(img, desired_size) for img in X_train]
X_test_resized = [cv2.resize(img, desired_size) for img in X_test]


# Konversi data gambar dan label menjadi array NumPy
X_train_array = np.array(X_train_resized)
X_test_array = np.array(X_test_resized)

# Reshape data untuk memenuhi format yang diperlukan oleh model (sesuai dengan input_shape)
# X_train_array = X_train_array.reshape(-1, 100, 100, 1)  # Ganti dimensi sesuai dengan ukuran gambar
# X_test_array = X_test_array.reshape(-1, 100, 100, 1)

# Normalisasi nilai piksel ke rentang [0, 1]
X_train_array = X_train_array / 255.0
X_test_array = X_test_array / 255.0

# Pelatihan model
history = model.fit(
    X_train_array, y_train_encoded,
    epochs=10,  # Ganti jumlah epoch sesuai kebutuhan
    batch_size=32,  # Ganti ukuran batch sesuai kebutuhan
    validation_data=(X_test_array, y_test_encoded)
)

# Evaluasi model
loss, accuracy = model.evaluate(X_test_array, y_test_encoded)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/10
2/2 [==============================] - 2s 721ms/step - loss: 1.3871 - accuracy: 0.2812 - val_loss: 1.3881 - val_accuracy: 0.1875
Epoch 2/10
2/2 [==============================] - 1s 600ms/step - loss: 1.3861 - accuracy: 0.2656 - val_loss: 1.3966 - val_accuracy: 0.1875
Epoch 3/10
2/2 [==============================] - 1s 473ms/step - loss: 1.3882 - accuracy: 0.2656 - val_loss: 1.4085 - val_accuracy: 0.1875
Epoch 4/10
2/2 [==============================] - 1s 498ms/step - loss: 1.3830 - accuracy: 0.2656 - val_loss: 1.4110 - val_accuracy: 0.1875
Epoch 5/10
2/2 [==============================] - 1s 488ms/step - loss: 1.3827 - accuracy: 0.2656 - val_loss: 1.4178 - val_accuracy: 0.1250
Epoch 6/10
2/2 [==============================] - 1s 504ms/step - loss: 1.3829 - accuracy: 0.2812 - val_loss: 1.4194 - val_accuracy: 0.1250
Epoch 7/10
2/2 [==============================] - 1s 449ms/step - loss: 1.3822 - accuracy: 0.2812 - val_loss: 1.4283 - val_accuracy: 0.1250
Epoch 8/10
2/2 [====

In [16]:

# Prediksi kelas untuk dataset pengujian
y_pred = model.predict(X_test_array)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai prediksi
y_pred_classes = np.argmax(y_pred, axis=1)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai label sebenarnya
y_true = np.argmax(y_test_encoded, axis=1)

# Menghitung akurasi
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Akurasi: {accuracy}")

# Membuat laporan klasifikasi
classification_rep = classification_report(y_true, y_pred_classes)
print("Laporan Klasifikasi:")
print(classification_rep)

# Menghitung dan menampilkan matriks kebingungan (confusion matrix)
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
print("Matriks Kebingungan:")
print(confusion_mtx)


1/1 [==============================] - 0s 71ms/step
Akurasi: 0.125
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.12      1.00      0.22         2

    accuracy                           0.12        16
   macro avg       0.03      0.25      0.06        16
weighted avg       0.02      0.12      0.03        16

Matriks Kebingungan:
[[0 0 0 3]
 [0 0 0 6]
 [0 0 0 5]
 [0 0 0 2]]


/home/ahmad/opencvPro/App-BPH-Manage/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ahmad/opencvPro/App-BPH-Manage/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ahmad/opencvPro/App-BPH-Manage/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [21]:
# Import library yang diperlukan

# Memuat model yang sudah dilatih
model = load_model('nama_model.h5')  # Ganti 'nama_model.h5' dengan nama model yang sudah Anda simpan

# Membaca gambar yang akan diprediksi
input_image_path = 'imgtest/diva/diva.21.png'  # Ganti dengan path gambar input Anda
input_image = cv2.imread(input_image_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)  # Ubah ke skala abu-abu jika diperlukan
input_image = cv2.resize(input_image, (100, 100))  # Sesuaikan dengan ukuran yang digunakan saat pelatihan
# cv2.imshow("Entah", input_image)

# cv2.waitKey(0)


input_image = input_image / 255.0  # Normalisasi

# Lakukan inferensi
input_image = np.expand_dims(input_image, axis=0)  # Menambahkan dimensi batch
predictions = model.predict(input_image)
# print(predicted_class)

# Mengambil kelas dengan probabilitas tertinggi sebagai prediksi
predicted_class = np.argmax(predictions, axis=1)

# Konversi indeks kelas menjadi label atau hasil yang sesuai dengan aplikasi Anda
# Contoh: jika Anda memiliki dictionary yang memetakan indeks ke label
# class_labels = {0: 'Label 0', 1: 'Label 1', 2: 'Label 2'} 
 # Ganti dengan label-label Anda

class_labels = {}
for i, isi in enumerate(os.listdir('img/')):
    class_labels[i] = isi

print(class_labels)
# print(predicted_class)
predicted_label = class_labels[predicted_class[0]]

# Cetak hasil prediksi
print(f'Hasil prediksi: {predicted_label}')


1/1 [==============================] - 0s 104ms/step
{0: 'dzikri', 1: 'diva', 2: 'uki', 3: 'arini'}
Hasil prediksi: arini


In [ ]:
print("i")

In [ ]:
# import os
# def ganti_nama(lama, baru):
#     os.rename(lama, baru)


# lsdir = os.listdir('img')
# print(lsdir)
# jumlah = 0
# hitung = 0
# for isi in lsdir:
#     lspath = os.listdir(f'img/{isi}')
#     print(len(lspath), isi)
#     jumlah += len(lspath)
#     hitung += 1



# lsdir = os.listdir('imgtest')
# print(lsdir)
# jumlah = 0
# hitung = 0
# for isi in lsdir:
#     lspath = os.listdir(f'imgtest/{isi}')
#     print(len(lspath), isi)
#     jumlah += len(lspath)
#     hitung += 1



    # jumlah = 1
    # for upath in lspath:

        # ganti_nama(f'img/{isi}/{upath}', f"img/{isi}/{isi}.{jumlah}.png")
        # jumlah += 1

    # jumlah = 1
    # for upath in lspath:

        # ganti_nama(f'img/{isi}/{upath}', f"img/{isi}/{isi}.{jumlah}.png")
        # jumlah += 1

In [ ]:
# path_dataset = 'dataset/'
# path_img = 'img/'

# face_cascade = cv.CascadeClassifier('cascade/face_detect.xml')
# eye_cascade = cv.CascadeClassifier('cascade/eye_detect.xml')

# while True:
#     img = cv.imread(f'{path_img}uki/uki.1.png')
#     gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#     wajah = face_cascade.detectMultiScale(gray, 1.3, 5)
#     for (x, y, w, h) in wajah:
#         cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
#         cv.imwrite(f"{path_dataset}uki.1.png", gray[y:y+h, x:x+w])
#     cv.imshow('img',img)
#     k = cv.waitKey(0)
#     if k == 27:
#         break

# cv.destroyAllWindows()

In [ ]:
# lsdir = os.listdir('img')
# print(lsdir)
# label_names = ["person1", "person2", "person3"]
# for labelid, labelname in enumerate(label_names):
#     print(f"id: {labelid} / nama: {labelname}")

In [ ]:
# path_dataset = 'dataset/'
# path_img = 'img/'

# face_cascade = cv.CascadeClassifier('cascade/face_detect.xml')
# eye_cascade = cv.CascadeClassifier('cascade/eye_detect.xml')

# for path_user in os.listdir(path_img):
#     if not os.path.exists(f'{path_dataset}{path_user}'):
#         os.mkdir(f'{path_dataset}{path_user}')
#     for foto in os.listdir(f"{path_img}{path_user}"):
#         img = cv.imread(f"{path_img}{path_user}/{foto}")
#         gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#         wajah = face_cascade.detectMultiScale(gray, 1.3, 5)
#         for (x, y, w, h) in wajah:
#             cv.imwrite(f'{path_dataset}{path_user}/{foto}', gray[y:y+h, x:x+w])


    
    


In [ ]:
print("i")

In [ ]:
# # import cv
# # import numpy as np

# # Path ke dataset wajah palsu (contoh)
# # path_dataset = "dataset/"

# # Inisialisasi array untuk menyimpan gambar dan label
# X = []  # Array gambar wajah
# y = []  # Array label

# # Mendefinisikan daftar nama label yang sesuai dengan setiap subfolder dalam dataset
# label_names = os.listdir(path_dataset)

# target_size = (128, 128)

# # Loop melalui setiap label (subfolder) dalam dataset
# for label_id, label_name in enumerate(label_names):
#     label_path = f"{path_dataset}{label_name}/"

#     # Loop melalui setiap gambar dalam subfolder
#     for image_filename in os.listdir(label_path):
#         image_path = os.path.join(label_path, image_filename)
#         print(image_path)

#         # Membaca gambar dengan OpenCV
#         image = cv.imread(image_path)
#         image = cv.resize(image, target_size)

#         # Mengubah gambar menjadi skala abu-abu jika perlu
#         if len(image.shape) > 2 and image.shape[2] == 3:
#             image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

#         # Normalisasi intensitas piksel ke rentang [0, 1]
#         image = image / 255.0

#         # Menambahkan gambar dan label ke dalam array
#         X.append(image)
#         y.append(label_id)

# # Mengubah array gambar dan label menjadi array NumPy
# X = np.array(X)
# y = np.array(y)

# # Sekarang Anda memiliki dataset wajah dalam 'X' (gambar) dan 'y' (label)
# # Anda dapat melanjutkan dengan melatih model pengenalan wajah dengan dataset ini.


In [79]:
face_cascade = cv2.CascadeClassifier('cascade/face_detect.xml')

def load_images_from_folder(folder):
    images = []
    labels = []
    
    label_encoder = LabelEncoder()
    for subfolder in os.listdir(folder):
        # subfolder_path = os.path.join(folder, subfolder)
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Ubah ke skala abu-abu jika perlu
                wajah = face_cascade.detectMultiScale(img, 1.3, 5)
                for (x, y, w, h) in wajah:
                    img = cv2.resize(img[y:y+h, x:x+w], (100, 100))
                    img = img / 255.0
    #                 cv2.imshow("gambar", img)
    #                 cv2.waitKey(0)
    # cv2.destroyAllWindows()
                    images.append(img)
                    print(subfolder)
                    labels.append(subfolder)  # Label adalah nama subfolder
    print(labels)
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    print(labels)
    return images, labels

dataset_folder = 'img/'
images, labels = load_images_from_folder(dataset_folder)

# load_images_from_folder(dataset_folder)


dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
dzikri
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
diva
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
uki
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
arini
['dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikri', 'dzikr

In [82]:


label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train_array = np.array(X_train)
X_test_array = np.array(X_test)

X_train_array = X_train_array.reshape(-1, 100, 100, 1)
X_test_array = X_test_array.reshape(-1, 100, 100, 1)

X_train_array = X_train_array / 255.0
X_test_array = X_test_array / 255.0

print(f"Jumlah data pelatihan: {len(X_train)}")
print(f"Jumlah data pengujian: {len(X_test)}")

Jumlah data pelatihan: 128
Jumlah data pengujian: 33


In [84]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# Inisialisasi model
model = models.Sequential()

# Tambahkan lapisan konvolusi pertama
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Tambahkan lapisan konvolusi kedua
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


# Tambahkan lapisan konvolusi ketiga (opsional)
# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))


# Lapisan Flatten untuk mengubah matriks menjadi vektor
model.add(layers.Flatten())

# Tambahkan lapisan fully connected pertama
model.add(layers.Dense(128, activation='relu'))

# Tambahkan lapisan fully connected kedua (output layer)
num_classes = 4  # Ganti dengan jumlah kelas yang sesuai dengan tugas Anda
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Jika ini adalah masalah klasifikasi
              metrics=['accuracy'])

# Tampilkan ringkasan model
model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d_36 (MaxPooli  (None, 49, 49, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_37 (MaxPooli  (None, 23, 23, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_11 (Flatten)        (None, 33856)             0         
                                                                 
 dense_22 (Dense)            (None, 128)             

                                                                 
 dense_23 (Dense)            (None, 4)                 516       
                                                                 
Total params: 4353028 (16.61 MB)
Trainable params: 4353028 (16.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [89]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Encode label pada y_test
label_binarizer = LabelBinarizer()
y_test_encoded = label_binarizer.fit_transform(y_test)

# Evaluasi model pada dataset pengujian
y_pred = model.predict(X_test_array)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai prediksi
y_pred_classes = np.argmax(y_pred, axis=1)

# Mengambil indeks kelas dengan probabilitas tertinggi sebagai label sebenarnya
y_true = np.argmax(y_test_encoded, axis=1)

# Menghitung akurasi
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Akurasi: {accuracy}")

# Menghitung presisi, recall, dan F1-score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Presisi: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Membuat laporan klasifikasi
classification_rep = classification_report(y_true, y_pred_classes)
print("Laporan Klasifikasi:")
print(classification_rep)

# Menghitung dan menampilkan matriks kebingungan (confusion matrix)
confusion_mtx = confusion_matrix(y_true, y_pred_classes)
print("Matriks Kebingungan:")
print(confusion_mtx)


2/2 [==============================] - 0s 10ms/step
Akurasi: 0.9090909090909091
Presisi: 0.9235209235209235
Recall: 0.9090909090909091
F1-Score: 0.9084787266605449
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       0.86      1.00      0.92         6
           2       1.00      0.80      0.89        10
           3       0.83      1.00      0.91        10

    accuracy                           0.91        33
   macro avg       0.92      0.91      0.91        33
weighted avg       0.92      0.91      0.91        33

Matriks Kebingungan:
[[ 6  1  0  0]
 [ 0  6  0  0]
 [ 0  0  8  2]
 [ 0  0  0 10]]


In [76]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [90]:
history = model.fit(
    X_train_array, y_train,
    epochs=10,
    batch_size=16,
    validation_data=(X_test_array, y_test)
)

loss, accuracy = model.evaluate(X_test_array, y_test)

Epoch 1/20


8/8 [==============================] - 1s 167ms/step - loss: 0.1049 - accuracy: 0.9844 - val_loss: 0.2599 - val_accuracy: 0.9091
Epoch 2/20
8/8 [==============================] - 1s 140ms/step - loss: 0.0923 - accuracy: 1.0000 - val_loss: 0.2611 - val_accuracy: 0.8788
Epoch 3/20
8/8 [==============================] - 1s 153ms/step - loss: 0.0675 - accuracy: 1.0000 - val_loss: 0.2193 - val_accuracy: 0.9091
Epoch 4/20
8/8 [==============================] - 1s 150ms/step - loss: 0.0687 - accuracy: 0.9922 - val_loss: 0.2102 - val_accuracy: 0.9091
Epoch 5/20
8/8 [==============================] - 1s 154ms/step - loss: 0.0521 - accuracy: 1.0000 - val_loss: 0.2444 - val_accuracy: 0.9091
Epoch 6/20
8/8 [==============================] - 1s 138ms/step - loss: 0.0486 - accuracy: 0.9922 - val_loss: 0.1869 - val_accuracy: 0.9394
Epoch 7/20
8/8 [==============================] - 1s 136ms/step - loss: 0.0406 - accuracy: 1.0000 - val_loss: 0.1910 - val_accuracy: 0.9394
Epoch 8/20
8/8 [===============

In [95]:

# Memuat gambar yang akan diuji
input_image_path = 'img/dzikri/dzikri.10.png'  # Ganti dengan path gambar input Anda
input_image = cv2.imread(input_image_path)  # Pastikan sesuai dengan preprocessing Anda
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY) # Normalisasi nilai piksel ke rentang [0, 1]

wajah = face_cascade.detectMultiScale(input_image, 1.1, 5) # Reshape gambar sesuai dengan format input model
for (x, y, w, h) in wajah:
    input_image = input_image[y:y+h, x:x+w]

input_image = cv2.resize(input_image, (100, 100))
input_image = input_image.astype('float32') / 255.0 # input_image = input_image / 255.0

input_image = np.expand_dims(input_image, axis=0)  # Menambahkan dimensi batch
predictions = model.predict(input_image) # Melakukan inferensi menggunakan model CNN

predicted_class = np.argmax(predictions, axis=1) # Mengambil kelas dengan probabilitas tertinggi sebagai prediksi
class_labels = {0:'arini', 1:'diva', 2:'dzikri',3:'uki'}  # Gantilah dengan label-label Anda

# Cetak hasil prediksi
print(f'Hasil prediksi: {class_labels[predicted_class[0]]}')


1/1 [==============================] - 0s 35ms/step
Hasil prediksi: dzikri


In [43]:
print(0)

0


In [41]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# Memuat model CNN yang telah dilatih
model_filename = 'nama_model.h5'  # Ganti dengan nama file model Anda
# model = load_model(model_filename)

# Memuat gambar yang akan diuji
input_image_path = 'imgtest/diva/diva.23.png'  # Ganti dengan path gambar input Anda
input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)  # Pastikan sesuai dengan preprocessing Anda

# Normalisasi nilai piksel ke rentang [0, 1]
input_image = input_image.astype('float32') / 255.0

input_image = cv2.resize(input_image, (100,100))

# Reshape gambar sesuai dengan format input model
input_image = np.expand_dims(input_image, axis=0)  # Menambahkan dimensi batch

# Melakukan inferensi menggunakan model CNN
predictions = model.predict(input_image)

# Mengambil kelas dengan probabilitas tertinggi sebagai prediksi
predicted_class = np.argmax(predictions, axis=1)

# Konversi indeks kelas menjadi label atau hasil yang sesuai dengan aplikasi Anda
# Contoh: jika Anda memiliki dictionary yang memetakan indeks ke label
# class_labels = {0: 'Label 0', 1: 'Label 1', 2: 'Label 2'} 
 # Ganti dengan label-label Anda

class_labels = {0:'dzikri', 1:'diva', 2:'uki',3:'arini'}  # Gantilah dengan label-label Anda
predicted_label = class_labels[predicted_class[0]]

# Cetak hasil prediksi
print(f'Hasil prediksi: {predicted_label}')


1/1 [==============================] - 0s 46ms/step
Hasil prediksi: arini


In [96]:

def uji_img(path_foto):
    # Memuat gambar yang akan diuji
    input_image_path = path_foto #'img/dzikri/dzikri.10.png'  # Ganti dengan path gambar input Anda
    input_image = cv2.imread(input_image_path)  # Pastikan sesuai dengan preprocessing Anda
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY) # Normalisasi nilai piksel ke rentang [0, 1]

    wajah = face_cascade.detectMultiScale(input_image, 1.1, 5) # Reshape gambar sesuai dengan format input model
    for (x, y, w, h) in wajah:
        input_image = input_image[y:y+h, x:x+w]

    input_image = cv2.resize(input_image, (100, 100))
    input_image = input_image.astype('float32') / 255.0 # input_image = input_image / 255.0

    input_image = np.expand_dims(input_image, axis=0)  # Menambahkan dimensi batch
    predictions = model.predict(input_image) # Melakukan inferensi menggunakan model CNN

    predicted_class = np.argmax(predictions, axis=1) # Mengambil kelas dengan probabilitas tertinggi sebagai prediksi
    class_labels = {0:'arini', 1:'diva', 2:'dzikri',3:'uki'}  # Gantilah dengan label-label Anda

    # Cetak hasil prediksi
    print(f'Hasil prediksi: {class_labels[predicted_class[0]]}')


In [104]:
path = 'imageTes/randomIMG'
for i, isi in enumerate(os.listdir(path)):
    print(i, isi)
for foto in os.listdir(path):
    uji_img(os.path.join(path, foto))

0 uki.19.png
1 dzikri.5.png
2 dzikri.4.png
3 dzikri.6.png
4 uki.20.png
5 uki.18.png
6 dzikri.3.png
7 uki.17.png
8 uki.16.png
1/1 [==============================] - 0s 29ms/step
Hasil prediksi: uki
1/1 [==============================] - 0s 30ms/step
Hasil prediksi: dzikri
1/1 [==============================] - 0s 33ms/step
Hasil prediksi: dzikri
1/1 [==============================] - 0s 29ms/step
Hasil prediksi: dzikri
1/1 [==============================] - 0s 28ms/step
Hasil prediksi: uki
1/1 [==============================] - 0s 55ms/step
Hasil prediksi: uki
1/1 [==============================] - 0s 25ms/step
Hasil prediksi: dzikri
1/1 [==============================] - 0s 29ms/step
Hasil prediksi: uki
1/1 [==============================] - 0s 30ms/step
Hasil prediksi: uki


In [57]:
path = 'img/dzikri/'
for foto in os.listdir(path):
    img_path = os.path.join(path, foto)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    wajah = face_cascade.detectMultiScale(gray, 1.1, 5)
    for (x, y, w, h) in wajah:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 1)
        print(img_path, end=" ")
    print(img_path)
    cv2.imshow('Hello',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

img/dzikri/dzikri.18.png img/dzikri/dzikri.18.png
img/dzikri/dzikri.12.png img/dzikri/dzikri.12.png
img/dzikri/dzikri.5.png img/dzikri/dzikri.5.png
img/dzikri/dzikri.19.png img/dzikri/dzikri.19.png
img/dzikri/dzikri.4.png img/dzikri/dzikri.4.png
img/dzikri/dzikri.7.png img/dzikri/dzikri.7.png
img/dzikri/dzikri.8.png img/dzikri/dzikri.8.png
img/dzikri/dzikri.10.png img/dzikri/dzikri.10.png
img/dzikri/dzikri.11.png img/dzikri/dzikri.11.png
img/dzikri/dzikri.44.png img/dzikri/dzikri.44.png
img/dzikri/dzikri.6.png img/dzikri/dzikri.6.png
img/dzikri/dzikri.13.png img/dzikri/dzikri.13.png
img/dzikri/dzikri.9.png img/dzikri/dzikri.9.png
img/dzikri/dzikri.3.png img/dzikri/dzikri.3.png
img/dzikri/dzikri.16.png img/dzikri/dzikri.16.png
img/dzikri/dzikri.17.png img/dzikri/dzikri.17.png
img/dzikri/dzikri.1.png img/dzikri/dzikri.1.png
img/dzikri/dzikri.2.png img/dzikri/dzikri.2.png
img/dzikri/dzikri.15.png img/dzikri/dzikri.15.png
img/dzikri/dzikri.20.png img/dzikri/dzikri.20.png
